In [3]:
import pandas as pd
import numpy as np
import pickle
from selenium import webdriver
import os
import time
from bs4 import BeautifulSoup

# 1. Load Data

In [21]:
_2019 = pd.read_excel('../data/external/2019-Report-Card-Public-Data-Set.xlsx', sheet_name=None)

In [ ]:
_2018 = pd.read_excel('../data/external/2018-Report-Card-Public-Data-Set.xlsx', sheet_name=None)

In [140]:
print(len(_2019['Financial'].columns))
print(len(_2018['Financial'].columns))

69
49


In [141]:
[x for x in _2018['Financial'].columns if '2016' not in x]

['RCDTS',
 'Type',
 'District',
 'City',
 'County',
 'District Type',
 'District Size',
 'EBF Tier',
 'EBF Capacity to Meet Expectations']

In [96]:
def get_columns_to_omit(df1, df2):
    col_intersection = set(df1.columns).intersection(set(df2.columns))
    col_intersection.remove('RCDTS')
    return list(col_intersection)

def merge_gen_fin_sat(dfs):
    gen = dfs['General']
    fin = dfs['Financial']
    sat = dfs['SAT']
    
    gen_fin = gen.merge(fin.drop(columns=get_columns_to_omit(gen, fin)), on="RCDTS", how="left")
    return gen_fin.merge(sat.drop(columns=get_columns_to_omit(gen_fin, sat)), on="RCDTS", how="left")
    

In [97]:
merged_2019 = merge_gen_fin_sat(_2019)
merged_2018 = merge_gen_fin_sat(_2018)

In [113]:
merged_2018.columns = [x.strip() for x in merged_2018.columns]
merged_2019.columns = [x.strip() for x in merged_2019.columns]

In [114]:
columns_difference = list(set(merged_2019.columns) - set(merged_2018.columns))
sorted(list(columns_difference))
# Glanced through columns that are different across both data sets and I believe I can drop them all
# except need to map school enrollment
len(columns_difference)

842

In [115]:
enroll_col_names_2018 = [x for x in merged_2018.columns if 'Enrollment' in x]
enroll_col_names_2019 = [x for x in merged_2019.columns if 'Enrollment' in x]
matched = enroll_col_names_2019[:-1]
matched.remove('% Student Enrollment - Children with Disabilities')

enroll_field_map_2018_to_2019 = dict(zip(enroll_col_names_2018,matched))

In [143]:
other_fields_2018_to_2019 = {
    'Total Students SAT Math Participation': '# Students SAT Math Participation',
    'Total Students SAT Math Participation %': '% Students SAT Math Participation',
    'Total Students SAT ELA Participation': '# Students SAT ELA Participation',
    'Total Students SAT ELA Participation %': '% SAT ELA Participation',
    'Five Essential Survey Student Response Rate %': 'Five Essential Survey Student Response Rate',
    'Five Essential Survey Teacher Response Rate %': 'Five Essential Survey Teacher Response Rate',
    'EBF Capacity to Meet Expectations': '% EBF Capacity to Meet Expectations'
}

def map_col_names_2018_2019(column_name_2018):
    if column_name_2018 in enroll_field_map_2018_to_2019:
        return enroll_field_map_2018_to_2019[column_name_2018]
    if column_name_2018 in other_fields_2018_to_2019:
        return other_fields_2018_to_2019[column_name_2018]
    return column_name_2018

merged_2018.columns = [map_col_names_2018_2019(x) for x in merged_2018.columns]

In [144]:
# _2017 = pd.read_csv('../data/external/rc17_assessment.txt', sep=';', header=None)
# 2017 data is raw, semi-colon delimited text files. Has a corresponding map to get column names
# Ignoring for now

In [117]:
print(merged_2018.shape, merged_2019.shape)

(4754, 605) (4738, 1125)


In [145]:
merged_2019['Year'] = 2019
merged_2018['Year'] = 2018
_18_19 = merged_2019.append(merged_2018, sort=False).reset_index()

In [146]:
_18_19.shape

(9492, 1441)

# 2. Explore Data

In [103]:
for col in _18_19.columns:
    print(col)

index
RCDTS
Type
School Name
District
City
County
District Type
District Size
School Type
Grades Served
Summative Designation
Summative Designation: Student Group(s)
Title 1 Status
State Senate District
State Representative District
# Student Enrollment
% Student Enrollment - White
% Student Enrollment - Black or African American
% Student Enrollment - Hispanic or Latino
% Student Enrollment - Asian
% Student Enrollment - Native Hawaiian or Other Pacific Islander
% Student Enrollment - American Indian or Alaska Native
% Student Enrollment - Two or More Races
% Student Enrollment - Children with Disabilities
% Student Enrollment - EL
% Student Enrollment - IEP
% Student Enrollment - Low Income
% Student Enrollment - Homeless
Total Number of School Days
% 8th Grade Passing Algebra 1
Student Attendance Rate
Student Mobility Rate
Student Mobility Rate - Male
Student Mobility Rate - Female
Student Mobility Rate - White
Student Mobility Rate - Black or African American
Student Mobility Rate 

#### Stuff it does have
- RCDTS - perhaps a unique identifier
- basic location data (county, city, district, school name)
- seems like it is for each school (3563 unique school name fields)
School       3872
District      865
Statewide       1
Name: Type, dtype: int64
- racial make up of the students
- extenuating circumstances (EL, IEP - broken down by type/need/dmo, Low Income, Homeless)
- attendence (# of days, attendence, truancy, chronic absenteeism)
- mobility rates for various demographics
- dropout rates for various demographics
- 4/5/6 year graduation rates
- Avg class size - all grades
- Min per day math/eng/social studies by grade (also phys ed)
- teacher info (total fte, race, gender, edu, novice, prov/short term license, retention, attendence, eval, salary, pupil teacher ratio)
- admin info (principal turnover, admin ratio, admin salary)
- 9th grade on track (by demo)
- student activies (CTE, advanced courses, AP/IB/Dual Credit - by demo and grade - enroll and test)
- enrolled in postsecondary (by demo, community college)
- survey data (leaders, involved families, ambitions instr, supportive, resp rate)
- Disciplinary info (total enrollment, suspensions, expulsions, arrests, law enf, violency
- demo info on "peer" districts 
- early childhood edu info
- state performance plan indicators (district, target, whether met taret)

#### Scrapable data has

Spending info
- Instructional Spending per Student ($)',
       'Operational Spending per Student ($)', 'Local Property Taxes (%)',
       'Other Local Funding (%)', 'Evidence-Based Funding (%)',
       'Other State Funding (%)', 'Federal Funding (%)', 'Total Revenue',
       (2018 on for below)
       'EBF Adequacy Target ($)',
       'EBF Capacity To Meet Funding Expectation (%)',
       'EBF Local Capacity Target ($)', 'EBF Real Receipts ($)',
       'EBF Local Capacity Target (%)', 'EBF Real Receipts (%)',
       'ESSA Designation'
- Feeder Schools (2018 on):
'ESSA Designation', 'ESSA Exemplary Schools',
       'ESSA Commendable Schools', 'ESSA Under Performing Schools',
       'ESSA Lowest Performing Schools', 'Has Feeder Schools',
- Overall Designation:


#### Stuff it doesn't have

- expenditure info
- growth in reading/math
- % evidence based funding formula
- feeder school ratings


In [147]:
relevant_columns = [
    'RCDTS',
    'Type',
    'School Name',
    'District',
    'City',
    'County',
    'District Type',
    'District Size',
    'School Type',
    'Grades Served',
    'Summative Designation',
    '# Student Enrollment',
    '% Student Enrollment - White',
    '% Student Enrollment - Black or African American',
    '% Student Enrollment - Hispanic or Latino',
    '% Student Enrollment - Asian',
    '% Student Enrollment - Native Hawaiian or Other Pacific Islander',
    '% Student Enrollment - American Indian or Alaska Native',
    '% Student Enrollment - Two or More Races',
#     '% Student Enrollment - Children with Disabilities', -> not in 2018
    '% Student Enrollment - EL',
    '% Student Enrollment - IEP',
    '% Student Enrollment - Low Income',
    '% Student Enrollment - Homeless',
    'Total Number of School Days',
    'Student Attendance Rate',
    'Student Mobility Rate',
    'High School Dropout Rate - Total',
    'High School 4-Year Graduation Rate - Total',
    'High School 6-Year Graduation Rate - Total',
    'Avg Class Size – All Grades',
    'Total Teacher FTE',
    'Avg Teaching Exp',
    'Bachelor Degree',
    'Masters Degree',
#     '% Novice Teachers', -> not in 2018
#     '% Novice Teachers - Low Poverty Schools',
    'Teacher Retention Rate',
    'Teacher Attendace Rate',
    'Teacher Evaluation Rate',
    'Principal Turnover within 6 Years',
    'Pupil Teacher Ratio - Elementary',
    'Pupil Teacher Ratio - High School',
    'Pupil Certified Staff Ratio',
    'Pupil Admin Ratio',
    'Teacher Avg Salary',
    'Admin Avg Salary',
    'Five Essential Survey Effective Leaders',
    'Five Essential Survey Leaders Level',
    'Five Essential Survey Collaborative Teachers',
    'Five Essential Survey Collaborative Teachers Level',
    'Five Essential Survey Involved Families',
    'Five Essential Survey Involved Families Level',
    'Five Essential Survey Supportive Environment',
    'Five Essential Survey Supportive Environment Level',
    'Five Essential Survey Ambitious Instruction',
    'Five Essential Survey Ambitious Instruction.1',
#     'Five Essential Survey Student Response Rate', -> not in 2018
#     'Five Essential Survey Teacher Response Rate',
#     'Five Essential Survey Schools with over 50% Response Rate',
    'Five Essential Survey Student Response Rate Median',
    'Five Essential Survey Teacher Response Rate Median',
#     '# CRDC Total Enrollment', -> nothing in 2018 with CRDC
    'EBF Tier',
    '% EBF Capacity to Meet Expectations',
#     '$ Site-Level Per-Pupil Expenditures - Federal',
#     '$ Site-level Per-Pupil Expenditures - State/Local',
#     '$ Site-level PEr-Pupil Expenditures - Subtotal',
#     '$ District Centralized Per-Pupil Expenditure - Federal',
#     '$ District Centralized Per-Pupil Expenditure - State/Local',
#     '$ District Centralized Per-Pupil Expenditure - Subtotal',
#     '$ Total Per-Pupil Expenditures - Federal',
#     '$ Total Per-Pupil Expenditures - State/Local',
#     '$ Total Per-Pupil Expenditures - Subtotal',
#     '$ Exclusions',
#     '$ Total Expenditures',
    'SAT Reading Total Students Level 1 %',
    'SAT Reading Total Students Level 2 %',
    'SAT Reading Total Students Level 3 %',
    'SAT Reading Total Students Level 4 %',
    'SAT Math Total Students Level 1 %',
    'SAT Math Total Students Level 2 %',
    'SAT Math Total Students Level 3 %',
    'SAT Math Total Students Level 4 %',
    '# Students SAT Math Participation',
    '% Students SAT Math Participation',
    '# Students SAT ELA Participation',
    '% SAT ELA Participation',
    'Year',
]

### Are they in both?

In [148]:
rel = set(relevant_columns)
col_in_2019 = set(merged_2019.columns)
col_in_2018 = set(merged_2018.columns)

In [149]:
rel - col_in_2019

set()

In [150]:
rel - col_in_2018

set()

In [151]:
subset = _18_19.loc[:, relevant_columns]

In [152]:
subset.shape

(9492, 71)

In [154]:
subset.to_csv('../data/interim/2018-19_relevant_columns.csv')